In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import torch.nn as nn
import torch
import pandas as pd
from bayesian_nnet.data import get_num_clases
import pytorch_lightning as pl
from bayesian_nnet.net_utils import initialize_pl_trainer, get_model_checkpoint_dir
from bayesian_nnet.data import IQDataModel
from bayesian_nnet.cnnlstm import AutoModClassifier
from bayesian_nnet.model_eval import init_snrid_accuracy, evaluate_model_predictions

In [48]:
checkpoint_file = "epoch=24-val_loss=0.00-other_metric=0.00.ckpt"

checkpoint_pth =\
    get_model_checkpoint_dir("frequentist_net").joinpath(checkpoint_file)

model = AutoModClassifier.load_from_checkpoint(checkpoint_pth)
trainer = pl.Trainer()
iqdata = IQDataModel(batch_size=256)

predictions = trainer.predict(model,
                              datamodule=iqdata)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                                                        | 0/? [00:00<?, ?it/s]

In [49]:
from bayesian_nnet.model_eval import ModelPredictionFormatter
from sklearn.metrics import classification_report, confusion_matrix
pred_fmt = ModelPredictionFormatter()
mode_conf_preds = pd.concat([pred_fmt(elem) for elem in predictions])

In [57]:
snrid_clf_report = dict()
snrid_conf_mat = dict()
modeids = list(mode_conf_preds.columns[:get_num_clases()])

for snrid in mode_conf_preds["snrid"].value_counts().keys():

    select_row = mode_conf_preds["snrid"] == snrid

    truemodeid =\
        mode_conf_preds.loc[select_row, "truemodeid"].values

    predictedmodeid =\
        mode_conf_preds.loc[select_row, "predictedmodeid"].values

    snrid_clf_report[snrid] =\
        classification_report(truemodeid,
                              predictedmodeid,
                              zero_division=0,
                              output_dict=True)

    cur_conf_mat =\
        confusion_matrix(truemodeid,
                         predictedmodeid,
                         normalize="true")

    snrid_conf_mat[snrid] =\
        pd.DataFrame(cur_conf_mat,
                     index=modeids,
                     columns=modeids)

In [60]:
{snrid:snrid_clf_report[snrid]["accuracy"]
 for snrid in snrid_clf_report.keys()}

{'snrminus5': 0.8333858267716535,
 'snr20': 0.9378005771080474,
 'snr5': 0.9222115075538412,
 'snr25': 0.9460338101430429,
 'snr15': 0.9429037520391517,
 'snrminus10': 0.6104881266490765,
 'snr10': 0.9359229747675962,
 'snr0': 0.9072648141948443}